In [1]:
import pandas as pd

In [6]:
coef = pd.read_csv('data/final_model.csv', header = None, names = ['names', 'coefficients'])
coef

,names,coefficients
0,Intercept,-83.860142
1,bedrooms,-0.056952
2,bathrooms,0.062707
3,sqft_living,0.485059
4,sqft_lot,0.066392
...,...,...
92,zipcode_98168,-0.057855
93,zipcode_98177,0.305928
94,zipcode_98178,0.086682
95,zipcode_98198,0.024175
